# Stable Spirograph Tutorial

This tutorial is based on `Spiro_Stable_From_Original.ipynb`.

Goal: explain the full workflow of music-reactive rendering with stable multi-entity collisions.

## Tutorial Outline
1. Config and runtime parameters
2. Audio analysis and frame-level feature mapping
3. Physics, invisible colliders, rendering, and video export

> Tip: keep `DURATION_SEC = 12` or `30` while tuning parameters.


In [ ]:
"""
Stable Music-Reactive Multi-Entity Spirograph Generator (Based on Spiro_Original)
-------------------------------------------------------------------------------
Changes vs. original:
1) 4 moving entities with circle collisions.
2) Collision radius is dynamic but clamped (prevents over-expansion lock).
3) Velocity is integrated with inertia (collision impulses persist), not hard-reset each frame.
4) Keeps original pipeline: audio analyze -> frame render -> ffmpeg encode.

Requirements:
  pip install numpy matplotlib librosa
  ffmpeg in PATH
"""

import os
import sys
import shutil
import subprocess
from dataclasses import dataclass

import numpy as np
import librosa
import matplotlib

matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.colors import hsv_to_rgb
from matplotlib.patches import Circle

# ==========================================
# 1. CONFIGURATION
# ==========================================

# --- I/O Paths ---
# You can replace AUDIO_PATH manually if needed.
AUDIO_PATH = r"C:\Users\12392\Desktop\Sprialgraph\11_20\luzhouyue.mp3"
AUDIO_FALLBACKS = [
    r"Larytta - Wonder Vendor (Live @ KEXP).mp3",
    r"luzhouyue.mp3",
]

OUTPUT_DIR = "frames_stable"
OUTPUT_FILENAME = "spirograph_stable_from_original.mp4"

# --- Video Settings ---
FPS = 30
DURATION_SEC = 30            # render first 30s (at 30fps ~= 900 frames)
WIDTH_INCH = 16
HEIGHT_INCH = 9
DPI = 100

# --- World / Physics ---
WORLD_W = 16.0
WORLD_H = 9.0
N_ENTITIES = 4
SEED = 7

VELOCITY_BASE = 1.8          # base center speed (world units / sec)
SPEED_LIMIT = 4.2             # hard cap for numerical stability
WALL_RESTITUTION = 0.98
COLLISION_RESTITUTION = 0.98
COLLISION_PASSES = 6
STEER_BLEND = 0.10            # lower => collision impulse persists longer

# Collider settings (invisible circles used by physics only)
COLLIDER_MIN = 0.95
COLLIDER_MAX = 2.20
COLLIDER_SCALE = 0.92         # near visual envelope for anti-overlap
COLLIDER_PAD = 0.10
DRAW_COLLIDER_DEBUG = False   # set True to visualize invisible colliders

# --- Spirograph Geometry Baselines (per entity) ---
R_STATOR_BASE = np.array([1.28, 1.18, 1.36, 1.22], dtype=np.float64)
R1_BASE = np.array([0.52, 0.46, 0.56, 0.49], dtype=np.float64)
R2_BASE = np.array([0.26, 0.22, 0.29, 0.24], dtype=np.float64)
D_PEN_BASE = np.array([0.40, 0.38, 0.43, 0.39], dtype=np.float64)

# Motion personality per entity
ENTITY_SPEED_FACTOR = np.array([0.94, 1.03, 1.10, 0.98], dtype=np.float64)
ENTITY_TURN_BIAS = np.array([-0.55, 0.70, -0.25, 0.45], dtype=np.float64)

NUM_POINTS = 1800
NUM_TURNS = 30


## Part 1: Audio Analysis (What this section does)

Responsibilities:
- Load and optionally trim audio (`DURATION_SEC`).
- Extract music features (tempo, loudness, brightness).
- Interpolate audio features to video frames so each frame has valid control values.

Core outputs:
- `tempo_scalar`: motion baseline
- `loudness`: affects speed, pen offset, and line width
- `brightness`: affects geometry scale and color


In [ ]:
# ==========================================
# 2. AUDIO ANALYSIS MODULE
# ==========================================

def resolve_audio_path() -> str:
    if os.path.exists(AUDIO_PATH):
        return AUDIO_PATH
    for p in AUDIO_FALLBACKS:
        if os.path.exists(p):
            return p
    return AUDIO_PATH


def analyze_audio(path, target_fps, duration_limit=None):
    """
    Loads audio, extracts features, and interpolates to video frame timestamps.
    If duration_limit is None or <= 0, use full track.
    """
    print(f"[Audio] Loading {path}...")

    y, sr = librosa.load(path, sr=None)
    full_duration = len(y) / sr

    if duration_limit is not None and float(duration_limit) > 0:
        duration_limit = float(duration_limit)
        if duration_limit < full_duration:
            max_samples = int(duration_limit * sr)
            y = y[:max_samples]
            print(f"[Audio] Truncating to {duration_limit:.2f}s for preview.")
        else:
            print("[Audio] duration_limit >= full duration, using full track.")
    else:
        print("[Audio] Using FULL track.")

    duration = len(y) / sr
    n_video_frames = int(np.floor(duration * target_fps))
    n_video_frames = max(n_video_frames, 1)
    print(f"[Audio] Effective Duration: {duration:.2f}s | Frames: {n_video_frames}")

    print("[Audio] Analyzing Tempo...")
    tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
    tempo_scalar = float(np.atleast_1d(tempo)[0]) / 120.0

    print("[Audio] Analyzing Loudness...")
    hop_length = 512
    rms = librosa.feature.rms(y=y, frame_length=2048, hop_length=hop_length)[0]

    print("[Audio] Analyzing Brightness...")
    centroid = librosa.feature.spectral_centroid(y=y, sr=sr, n_fft=2048, hop_length=hop_length)[0]

    times_audio = librosa.frames_to_time(np.arange(len(rms)), sr=sr, hop_length=hop_length)

    def normalize_and_smooth(data, window=15):
        data = np.nan_to_num(data)
        d_min, d_max = np.min(data), np.max(data)
        if d_max - d_min > 1e-12:
            norm = (data - d_min) / (d_max - d_min)
        else:
            norm = np.zeros_like(data)
        kernel = np.ones(window, dtype=np.float64) / float(window)
        return np.convolve(norm, kernel, mode='same')

    loud_norm = normalize_and_smooth(rms)
    bright_norm = normalize_and_smooth(centroid)

    times_video = np.linspace(0, duration, n_video_frames, endpoint=False)
    loud_video = np.interp(times_video, times_audio, loud_norm)
    bright_video = np.interp(times_video, times_audio, bright_norm)

    return {
        "duration": duration,
        "n_frames": n_video_frames,
        "tempo_scalar": tempo_scalar,
        "loudness": loud_video,
        "brightness": bright_video,
    }


## Part 2: Physics and Invisible Colliders (What this section does)

Responsibilities:
- Define `Entity` state (position, velocity, collider radius, phases).
- Handle wall bounce and entity-entity collisions.
- Use invisible circles for collision solving to keep the simulation stable and fast.

Why invisible colliders:
- Curve-vs-curve geometric collision is expensive and unstable.
- Circle colliders give robust motion while keeping the visual curves complex.

Debug:
- Set `DRAW_COLLIDER_DEBUG = True` to visualize collider circles for tuning.


In [ ]:
# ==========================================
# 3. PHYSICS & MATH
# ==========================================

@dataclass
class Entity:
    pos: np.ndarray
    vel: np.ndarray
    collider_radius: float
    phase1: float
    phase2: float


def _clamp_speed(v: np.ndarray, max_speed: float) -> np.ndarray:
    speed = float(np.linalg.norm(v))
    if speed > max_speed and speed > 1e-12:
        return v * (max_speed / speed)
    return v


def init_entities(n, seed=7):
    rng = np.random.default_rng(seed)
    entities = []

    x_min, x_max = 1.2, WORLD_W - 1.2
    y_min, y_max = 1.2, WORLD_H - 1.2

    base_r = 1.00
    for _ in range(n):
        pos = None
        for _attempt in range(600):
            cand = np.array([
                rng.uniform(x_min, x_max),
                rng.uniform(y_min, y_max),
            ], dtype=np.float64)
            if all(np.linalg.norm(cand - e.pos) > (base_r + e.collider_radius + 0.4) for e in entities):
                pos = cand
                break
        if pos is None:
            # Fallback ring layout
            a = 2.0 * np.pi * (len(entities) / max(n, 1))
            pos = np.array([
                WORLD_W * 0.5 + 2.4 * np.cos(a),
                WORLD_H * 0.5 + 1.8 * np.sin(a),
            ], dtype=np.float64)

        angle = rng.uniform(0.0, 2.0 * np.pi)
        speed = rng.uniform(0.9, 1.4)
        vel = np.array([np.cos(angle), np.sin(angle)], dtype=np.float64) * speed

        entities.append(
            Entity(
                pos=pos,
                vel=vel,
                collider_radius=base_r,
                phase1=rng.uniform(0.0, 2.0 * np.pi),
                phase2=rng.uniform(0.0, 2.0 * np.pi),
            )
        )

    return entities


def resolve_wall_bounce(entity: Entity):
    r = float(entity.collider_radius + 0.03)

    if entity.pos[0] < r:
        entity.pos[0] = r
        entity.vel[0] = abs(entity.vel[0]) * WALL_RESTITUTION
    elif entity.pos[0] > WORLD_W - r:
        entity.pos[0] = WORLD_W - r
        entity.vel[0] = -abs(entity.vel[0]) * WALL_RESTITUTION

    if entity.pos[1] < r:
        entity.pos[1] = r
        entity.vel[1] = abs(entity.vel[1]) * WALL_RESTITUTION
    elif entity.pos[1] > WORLD_H - r:
        entity.pos[1] = WORLD_H - r
        entity.vel[1] = -abs(entity.vel[1]) * WALL_RESTITUTION


def resolve_entity_collisions(entities, restitution=0.98, slop=1e-3, percent=0.92):
    n = len(entities)

    for _ in range(COLLISION_PASSES):
        for i in range(n):
            for j in range(i + 1, n):
                a = entities[i]
                b = entities[j]

                delta = b.pos - a.pos
                dist = float(np.linalg.norm(delta))
                min_dist = float(a.collider_radius + b.collider_radius)

                if dist >= min_dist:
                    continue

                if dist < 1e-9:
                    normal = np.array([1.0, 0.0], dtype=np.float64)
                    dist = 1e-9
                else:
                    normal = delta / dist

                # Velocity impulse (equal masses)
                rv = b.vel - a.vel
                vel_along_normal = float(np.dot(rv, normal))
                if vel_along_normal < 0.0:
                    impulse_mag = -(1.0 + restitution) * vel_along_normal / 2.0
                    impulse = impulse_mag * normal
                    a.vel -= impulse
                    b.vel += impulse

                # Positional correction
                penetration = min_dist - dist
                corr_mag = max(penetration - slop, 0.0) * percent / 2.0
                correction = corr_mag * normal
                a.pos -= correction
                b.pos += correction

                # Small separation kick makes collisions visually clearer.
                sep_boost = 0.03 + 0.05 * max(penetration, 0.0)
                a.vel -= sep_boost * normal
                b.vel += sep_boost * normal


def compute_multi_rotor_points(cx, cy, R, r1, r2, d, num_points, num_turns, phase1, phase2):
    """Vectorized 3-stage spirograph with per-entity phase offsets."""
    t_vals = np.linspace(0, num_turns * 2 * np.pi, num_points)

    p1 = t_vals + phase1
    p2 = t_vals + phase2

    c1_x = cx + (R - r1) * np.cos(p1)
    c1_y = cy + (R - r1) * np.sin(p1)

    theta1 = -(R / r1) * p1

    c2_x = c1_x + (r1 - r2) * np.cos(theta1)
    c2_y = c1_y + (r1 - r2) * np.sin(theta1)

    theta2 = -(r1 / r2) * (theta1 + p2)

    pen_angle = theta1 + theta2
    px = c2_x + d * np.cos(pen_angle)
    py = c2_y + d * np.sin(pen_angle)

    return px, py


def color_from_features(bright, loud, tempo_scalar, entity_idx):
    # Entity hue offsets keep layers separable
    hue = (0.58 + 0.14 * entity_idx + 0.34 * bright + 0.05 * tempo_scalar) % 1.0
    sat = np.clip(0.45 + 0.45 * loud, 0.25, 1.0)
    val = np.clip(0.65 + 0.30 * loud, 0.35, 1.0)
    return hsv_to_rgb((hue, sat, val))


## Part 3: Render and Export (What this section does)

Responsibilities:
- Per-frame pipeline: feature update -> physics step -> draw curves -> save frame.
- Encode frames into mp4 with FFmpeg and mux original audio.

Common tuning knobs:
- Reduce overlap: increase `COLLIDER_SCALE` or `COLLIDER_MIN`.
- Stronger collision feel: increase `COLLISION_RESTITUTION`, decrease `STEER_BLEND`.
- Faster render: decrease `NUM_POINTS` and `NUM_TURNS`.


In [ ]:
# ==========================================
# 4. MAIN RENDER LOOP
# ==========================================

def main():
    if shutil.which("ffmpeg") is None:
        print("ERROR: ffmpeg is not installed or not in PATH.")
        return

    audio_path = resolve_audio_path()
    if not os.path.exists(audio_path):
        print(f"ERROR: audio file not found: {audio_path}")
        return

    if os.path.exists(OUTPUT_DIR):
        shutil.rmtree(OUTPUT_DIR)
    os.makedirs(OUTPUT_DIR)

    data = analyze_audio(audio_path, FPS, DURATION_SEC)

    fig = plt.figure(figsize=(WIDTH_INCH, HEIGHT_INCH), dpi=DPI)
    ax = fig.add_axes([0, 0, 1, 1])
    ax.set_facecolor("black")

    entities = init_entities(N_ENTITIES, seed=SEED)
    dt = 1.0 / FPS
    n_frames = data["n_frames"]

    print(f"\n[Render] Starting {n_frames} frames...")
    print(f"[Render] Output directory: {os.path.abspath(OUTPUT_DIR)}")

    for i in range(n_frames):
        loud = float(data["loudness"][i])
        bright = float(data["brightness"][i])
        tempo_s = float(data["tempo_scalar"])

        speed_scale = (0.55 + 0.80 * tempo_s) * (0.45 + 1.45 * loud)

        frame_params = []

        for k, ent in enumerate(entities):
            # --- Geometry from base (no accumulation bug) ---
            R_curr = R_STATOR_BASE[k] * (0.82 + 0.52 * bright)
            r1_curr = R1_BASE[k] * (0.90 + 0.18 * np.sin(i * 0.045 * (1.0 + 0.2 * k)))
            r2_curr = R2_BASE[k] * (0.90 + 0.18 * np.cos(i * 0.031 * (1.0 + 0.15 * k)))
            d_curr = D_PEN_BASE[k] * (0.52 + 1.60 * loud)

            # Numeric guard rails
            R_curr = float(np.clip(R_curr, 0.65 * R_STATOR_BASE[k], 1.45 * R_STATOR_BASE[k]))
            r1_curr = float(np.clip(r1_curr, 0.65 * R1_BASE[k], 1.35 * R1_BASE[k]))
            r2_curr = float(np.clip(r2_curr, 0.60 * R2_BASE[k], 1.35 * R2_BASE[k]))
            d_curr = float(np.clip(d_curr, 0.35 * D_PEN_BASE[k], 2.10 * D_PEN_BASE[k]))

            if r2_curr >= 0.93 * r1_curr:
                r2_curr = 0.93 * r1_curr

            # Dynamic collider, clamped to avoid lock-up in finite world.
            extent = (R_curr - r2_curr) + d_curr
            c_radius = COLLIDER_SCALE * extent + COLLIDER_PAD + 0.08 * loud
            ent.collider_radius = float(np.clip(c_radius, COLLIDER_MIN, COLLIDER_MAX))

            # Desired motion direction = smooth turn from current velocity
            v_norm = float(np.linalg.norm(ent.vel))
            if v_norm < 1e-8:
                curr_dir = np.array([1.0, 0.0], dtype=np.float64)
            else:
                curr_dir = ent.vel / v_norm

            omega = (
                0.35 * (tempo_s - 1.0)
                + 1.20 * (loud - 0.5)
                + 0.80 * (bright - 0.5)
                + 0.65 * ENTITY_TURN_BIAS[k]
                + 0.18 * np.sin(0.013 * i + k)
            )

            ang = omega * dt
            ca, sa = np.cos(ang), np.sin(ang)
            desired_dir = np.array([
                ca * curr_dir[0] - sa * curr_dir[1],
                sa * curr_dir[0] + ca * curr_dir[1],
            ], dtype=np.float64)

            target_speed = VELOCITY_BASE * speed_scale * ENTITY_SPEED_FACTOR[k]
            desired_vel = desired_dir * target_speed

            # Keep inertia so collision impulses are not erased next frame.
            ent.vel = (1.0 - STEER_BLEND) * ent.vel + STEER_BLEND * desired_vel
            ent.vel = _clamp_speed(ent.vel, SPEED_LIMIT)

            ent.pos = ent.pos + ent.vel * dt
            resolve_wall_bounce(ent)

            # Independent phase clocks per entity.
            ent.phase1 += dt * (1.05 + 0.90 * tempo_s + 0.45 * loud)
            ent.phase2 += dt * (0.80 + 0.70 * tempo_s + 0.55 * bright)

            frame_params.append((R_curr, r1_curr, r2_curr, d_curr))

        # Resolve inter-entity collisions after all moved.
        resolve_entity_collisions(entities, restitution=COLLISION_RESTITUTION)

        # Collision correction can push entities into walls.
        for ent in entities:
            resolve_wall_bounce(ent)
            ent.vel = _clamp_speed(ent.vel, SPEED_LIMIT)

        # --- Draw ---
        ax.clear()
        ax.set_xlim(0, WORLD_W)
        ax.set_ylim(0, WORLD_H)
        ax.axis("off")

        for k, ent in enumerate(entities):
            R_curr, r1_curr, r2_curr, d_curr = frame_params[k]

            xs, ys = compute_multi_rotor_points(
                ent.pos[0], ent.pos[1],
                R_curr, r1_curr, r2_curr, d_curr,
                NUM_POINTS, NUM_TURNS,
                ent.phase1, ent.phase2,
            )

            color = color_from_features(bright, loud, tempo_s, k)
            lw = float(np.clip(0.9 + 3.8 * loud + 0.25 * k, 0.8, 5.5))
            alpha = float(np.clip(0.35 + 0.50 * loud + 0.07 * k, 0.25, 0.95))
            ax.plot(xs, ys, color=color, linewidth=lw, alpha=alpha)

            if DRAW_COLLIDER_DEBUG:
                circ = Circle((ent.pos[0], ent.pos[1]), ent.collider_radius, fill=False, ec=color, lw=0.8, ls='--', alpha=0.20)
                ax.add_patch(circ)

        filename = os.path.join(OUTPUT_DIR, f"frame_{i:05d}.png")
        fig.savefig(filename, facecolor="black")

        if i % 20 == 0:
            sys.stdout.write(f"\rProcessing Frame {i}/{n_frames}")
            sys.stdout.flush()

    plt.close(fig)
    print("\n[Render] Frame generation complete.")

    print("[FFmpeg] Encoding video...")
    cmd = [
        "ffmpeg", "-y",
        "-framerate", str(FPS),
        "-i", os.path.join(OUTPUT_DIR, "frame_%05d.png"),
        "-i", audio_path,
        "-c:v", "libx264",
        "-pix_fmt", "yuv420p",
        "-c:a", "aac",
        "-b:a", "192k",
        "-shortest",
        OUTPUT_FILENAME,
    ]

    print(f"[FFmpeg] Command: {' '.join(cmd)}")
    try:
        subprocess.run(cmd, check=True)
        print("\n" + "=" * 52)
        print(f"DONE! Video saved to: {os.path.abspath(OUTPUT_FILENAME)}")
        print("=" * 52)
    except subprocess.CalledProcessError as e:
        print(f"[FFmpeg] Error: {e}")


# Run render (execute this cell or call main() manually)
# main()


## Run

After parameter tuning, run the next cell to render the video:


In [ ]:
main()
